In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('../../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0,1
1,36865,ESPORTSTMNT05_2520933,19,21,1,22,25,24,20,21,...,8,0.38,3.2,21,0.57,3.3,6,0.83,4.1,0
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,0
3,36867,ESPORTSTMNT02_2557426,20,0,21,20,21,5,23,1,...,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0,0
4,36868,ESPORTSTMNT02_2557463,19,1,2,1,2,4,20,24,...,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4110,44383,ESPORTSTMNT02_3047878,0,20,21,1,24,21,22,23,...,21,0.62,6.1,13,0.69,4.8,11,0.82,10.4,0
4111,44384,ESPORTSTMNT02_3047898,19,20,2,20,3,6,24,5,...,36,0.75,5.6,38,0.71,5.5,8,0.88,4.3,0
4112,44385,ESPORTSTMNT02_3047499,0,2,24,1,22,21,20,4,...,43,0.67,7.4,64,0.72,5.5,36,0.53,2.3,1
4113,44386,ESPORTSTMNT02_3057927,19,2,5,4,2,22,7,1,...,8,0.50,3.5,22,0.55,4.7,3,0.33,3.1,1


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,19,1,2,1,3,1,1.00,2.0,1,1.00,...,0.0,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0
1,19,21,1,22,25,5,0.60,5.6,2,0.00,...,3.5,8,0.38,3.2,21,0.57,3.3,6,0.83,4.1
2,19,2,20,24,2,0,0.00,0.0,1,1.00,...,0.0,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2
3,20,0,21,20,21,23,0.61,3.6,5,0.20,...,4.2,15,0.67,3.9,34,0.74,7.3,2,0.00,2.0
4,19,1,2,1,2,3,1.00,9.8,2,1.00,...,4.7,5,0.40,2.4,2,1.00,24.0,1,1.00,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4110,0,20,21,1,24,30,0.67,3.4,21,0.81,...,4.4,21,0.62,6.1,13,0.69,4.8,11,0.82,10.4
4111,19,20,2,20,3,23,0.65,4.2,0,0.00,...,3.6,36,0.75,5.6,38,0.71,5.5,8,0.88,4.3
4112,0,2,24,1,22,25,0.80,3.7,9,0.67,...,1.8,43,0.67,7.4,64,0.72,5.5,36,0.53,2.3
4113,19,2,5,4,2,18,0.50,2.5,3,0.67,...,4.0,8,0.50,3.5,22,0.55,4.7,3,0.33,3.1


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [5]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [1], 
              'gamma': [0.01],
              'kernel': ['sigmoid']} 
  
grid = GridSearchCV(SVC(random_state=42),param_grid,refit=True,cv=3,n_jobs=-1)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [6]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 1, 'gamma': 0.01, 'kernel': 'sigmoid'}
SVC(C=1, gamma=0.01, kernel='sigmoid', random_state=42)


In [9]:
SVCOpt = SVC(random_state=42,C=1,gamma=0.01,kernel='sigmoid',max_iter=50000)
SVCOpt.fit(X_train, y_train)

scores_list = []
scores_list.append({
    'Balanced Accuracy': balanced_accuracy_score(y_test,SVCOpt.predict(X_test)),
    'Precision':  precision_score(y_test,SVCOpt.predict(X_test)),
    'Recall': recall_score(y_test,SVCOpt.predict(X_test)),
    'F1-Score': f1_score(y_test,SVCOpt.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,SVCOpt.predict(X_test))
})
scores = pd.DataFrame(scores_list)

In [11]:
scores

,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,0.711935,0.720764,0.71564,0.718193,0.711935
